In [54]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [55]:
data_gold_usd = pd.read_csv("csv files/XAU_USD.csv")
data_gold_usd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2857 entries, 0 to 2856
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2857 non-null   object 
 1   Price     2857 non-null   object 
 2   Open      2857 non-null   object 
 3   High      2857 non-null   object 
 4   Low       2857 non-null   object 
 5   Vol.      0 non-null      float64
 6   Change %  2857 non-null   object 
dtypes: float64(1), object(6)
memory usage: 156.4+ KB


In [15]:
df = data_gold_usd.drop(['Open','High','Low','Vol.','Change %'], axis=1)

In [16]:
df['Price'] = df['Price'].apply(lambda x: float(x.split()[0].replace(',', ''))/1000)
df

,Date,Price
0,12/19/2022,1.79238
1,12/18/2022,1.79114
2,12/16/2022,1.79234
3,12/15/2022,1.77666
4,12/14/2022,1.80727
...,...,...
2852,01/06/2012,1.61685
2853,01/05/2012,1.62300
2854,01/04/2012,1.61240
2855,01/03/2012,1.60414


## LSTM

In [23]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df['Price']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[1.79238 1.79114 1.79234 1.77666 1.80727] 1.81029
[1.79114 1.79234 1.77666 1.80727 1.81029] 1.78115
[1.79234 1.77666 1.80727 1.81029 1.78115] 1.7966300000000002
[1.77666 1.80727 1.81029 1.78115 1.79663] 1.78923
[1.80727 1.81029 1.78115 1.79663 1.78923] 1.78622
[1.81029 1.78115 1.79663 1.78923 1.78622] 1.77091
[1.78115 1.79663 1.78923 1.78622 1.77091] 1.7683900000000001
[1.79663 1.78923 1.78622 1.77091 1.76839] 1.79782
[1.78923 1.78622 1.77091 1.76839 1.79782] 1.80289
[1.78622 1.77091 1.76839 1.79782 1.80289] 1.76845
[1.77091 1.76839 1.79782 1.80289 1.76845] 1.74973
[1.76839 1.79782 1.80289 1.76845 1.74973] 1.74075
[1.79782 1.80289 1.76845 1.74973 1.74075] 1.75614
[1.80289 1.76845 1.74973 1.74075 1.75614] 1.75496
[1.76845 1.74973 1.74075 1.75614 1.75496] 1.74928
[1.74973 1.74075 1.75614 1.75496 1.74928] 1.74024
[1.74075 1.75614 1.75496 1.74928 1.74024] 1.7378099999999999
[1.75614 1.75496 1.74928 1.74024 1.73781] 1.74974
[1.75496 1.74928 1.74024 1.73781 1.74974] 1.76087
[1.74928 1.74024 

In [24]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)


Epoch 1/200
90/90 [==============================] - 2s 5ms/step - loss: 0.1996
Epoch 2/200
90/90 [==============================] - 0s 5ms/step - loss: 5.7044e-04
Epoch 3/200
90/90 [==============================] - 0s 5ms/step - loss: 5.7422e-04
Epoch 4/200
90/90 [==============================] - 0s 5ms/step - loss: 5.7089e-04
Epoch 5/200
90/90 [==============================] - 0s 5ms/step - loss: 6.2135e-04
Epoch 6/200
90/90 [==============================] - 0s 5ms/step - loss: 5.7712e-04
Epoch 7/200
90/90 [==============================] - 0s 5ms/step - loss: 5.8029e-04
Epoch 8/200
90/90 [==============================] - 0s 5ms/step - loss: 5.8422e-04
Epoch 9/200
90/90 [==============================] - 0s 5ms/step - loss: 5.8773e-04
Epoch 10/200
90/90 [==============================] - 0s 6ms/step - loss: 6.2768e-04
Epoch 11/200
90/90 [==============================] - 0s 5ms/step - loss: 5.9038e-04
Epoch 12/200
90/90 [==============================] - 0s 5ms/step - loss: 5.94

In [29]:
# demonstrate prediction
x_input = array([[df['Price'][0],df['Price'][1],df['Price'][2],df['Price'][3],df['Price'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

## Daily

In [30]:
print(f"Prediction: {round(yhat[0][0]*1000,2)} $ Gold/USD") # onluk olarak yukarı yuvarlandı.

Prediction: 1816.63 $ Gold/USD


## Weekly

In [38]:
x_input = array([df['Price'][0], df['Price'][1], df['Price'][2], df['Price'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Price'][0], df['Price'][1], df['Price'][2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Price'][0], df['Price'][1],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Price'][0], float(yhat_4),float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)


yhat = float(yhat)*1000
yhat_2 = float(yhat_2)*1000
yhat_3 = float(yhat_3)*1000
yhat_4 = float(yhat_4)*1000
yhat_5 = float(yhat_5)*1000


In [39]:
# onluk olarak yukarı yuvarlandı.
print(f"Pazartesi: {round(yhat,3)} Salı: {round(yhat_2,3)} Carsamba: {round(yhat_3,3)} Persembe: {round(yhat_4,3)} Cuma: {round(yhat_5,3)}  ₺ Pound/TRY")


Pazartesi: 1816.627 Salı: 1826.527 Carsamba: 1822.693 Persembe: 1823.159 Cuma: 1823.344  ₺ Pound/TRY


## Monthly

In [41]:
import array
exchange = array.array('f', [])

for i in range(len(df)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df['Price'][i])


In [42]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.79238 1.81029 1.77091 1.74973 1.74024] 1.778249979019165
[1.81029 1.77091 1.74973 1.74024 1.77825] 1.7125200033187866
[1.77091 1.74973 1.74024 1.77825 1.71252] 1.6475000381469727
[1.74973 1.74024 1.77825 1.71252 1.6475 ] 1.652519941329956
[1.74024 1.77825 1.71252 1.6475  1.65252] 1.6518299579620361
[1.77825 1.71252 1.6475  1.65252 1.65183] 1.665310025215149
[1.71252 1.6475  1.65252 1.65183 1.66531] 1.7262400388717651
[1.6475  1.65252 1.65183 1.66531 1.72624] 1.6288000345230103
[1.65252 1.65183 1.66531 1.72624 1.6288 ] 1.6631100177764893
[1.65183 1.66531 1.72624 1.6288  1.66311] 1.701550006866455
[1.66531 1.72624 1.6288  1.66311 1.70155] 1.7011499404907227
[1.72624 1.6288  1.66311 1.70155 1.70115] 1.7235499620437622
[1.6288  1.66311 1.70155 1.70115 1.72355] 1.74767005443573
[1.66311 1.70155 1.70115 1.72355 1.74767] 1.7755399942398071
[1.70155 1.70115 1.72355 1.74767 1.77554] 1.7940800189971924
[1.70115 1.72355 1.74767 1.77554 1.79408] 1.7597500085830688
[1.72355 1.74767 1.77554 1.794

In [43]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
18/18 [==============================] - 1s 7ms/step - loss: 0.8215
Epoch 2/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0369
Epoch 3/300
18/18 [==============================] - 0s 8ms/step - loss: 0.0094
Epoch 4/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0034
Epoch 5/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0028
Epoch 6/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0024
Epoch 7/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0024
Epoch 8/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0025
Epoch 9/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0025
Epoch 10/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0025
Epoch 11/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0024
Epoch 12/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0024
Epoch 13/300
18/18 [=================

In [53]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Month -> Prediction: {round(float(yhat[0][0]*1000),3)}  ₺ Gold/USD ")


Next Month -> Prediction: 1726.263  ₺ Gold/USD 


## Yearly

In [46]:
import array
exchange_2 = array.array('f', [])

for i in range(len(df)):
    if i % 30 == 0:  # it equals to weekly
        exchange_2.append(df['Price'][i])


In [47]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange_2
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.79238 1.71252 1.6288  1.77554 1.76435] 1.8659000396728516
[1.71252 1.6288  1.77554 1.76435 1.8659 ] 1.9665900468826294
[1.6288  1.77554 1.76435 1.8659  1.96659] 1.9432200193405151
[1.77554 1.76435 1.8659  1.96659 1.94322] 1.813539981842041
[1.76435 1.8659  1.96659 1.94322 1.81354] 1.7840399742126465
[1.8659  1.96659 1.94322 1.81354 1.78404] 1.7926299571990967
[1.96659 1.94322 1.81354 1.78404 1.79263] 1.8042700290679932
[1.94322 1.81354 1.78404 1.79263 1.80427] 1.809939980506897
[1.81354 1.78404 1.79263 1.80427 1.80994] 1.7785099744796753
[1.78404 1.79263 1.80427 1.80994 1.77851] 1.836940050125122
[1.79263 1.80427 1.80994 1.77851 1.83694] 1.6848000288009644
[1.80427 1.80994 1.77851 1.83694 1.6848 ] 1.7943099737167358
[1.80994 1.77851 1.83694 1.6848  1.79431] 1.9493499994277954
[1.77851 1.83694 1.6848  1.79431 1.94935] 1.8075100183486938
[1.83694 1.6848  1.79431 1.94935 1.80751] 1.890820026397705
[1.6848  1.79431 1.94935 1.80751 1.89082] 1.9704899787902832
[1.79431 1.94935 1.80751 1.8

In [48]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange_2[0], exchange_2[1], exchange_2[2], exchange_2[3], exchange_2[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
3/3 [==============================] - 1s 8ms/step - loss: 1.7644
Epoch 2/300
3/3 [==============================] - 0s 7ms/step - loss: 1.1148
Epoch 3/300
3/3 [==============================] - 0s 7ms/step - loss: 0.5384
Epoch 4/300
3/3 [==============================] - 0s 7ms/step - loss: 0.0932
Epoch 5/300
3/3 [==============================] - 0s 6ms/step - loss: 0.1811
Epoch 6/300
3/3 [==============================] - 0s 7ms/step - loss: 0.1339
Epoch 7/300
3/3 [==============================] - 0s 7ms/step - loss: 0.0189
Epoch 8/300
3/3 [==============================] - 0s 8ms/step - loss: 0.0581
Epoch 9/300
3/3 [==============================] - 0s 6ms/step - loss: 0.0785
Epoch 10/300
3/3 [==============================] - 0s 6ms/step - loss: 0.0483
Epoch 11/300
3/3 [==============================] - 0s 8ms/step - loss: 0.0150
Epoch 12/300
3/3 [==============================] - 0s 6ms/step - loss: 0.0226
Epoch 13/300
3/3 [==============================] - 0s 10ms/s

In [51]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Year -> Prediction: {round(float(yhat[0][0]*1000),3)}  ₺ GOLD/USD ")


Next Year -> Prediction: 1726.263  ₺ GOLD/USD 
